## Differences between Spark objects:
<img alt="memory usage" src="https://databricks.com/wp-content/uploads/2016/07/memory-usage-when-caching-datasets-vs-rdds.png" width=600>

<img alt="type safety" src="https://databricks.com/wp-content/uploads/2016/07/sql-vs-dataframes-vs-datasets-type-safety-spectrum.png" width=600>

### Use an RDD when:
[quoted from databricks](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html)

> - you want low-level transformation and actions and control on your dataset;
> - your data is unstructured, such as media streams or streams of text;
> - you want to manipulate your data with functional programming constructs than domain specific expressions;
> - you don’t care about imposing a schema, such as columnar format, while processing or accessing data attributes by name or column

### Use a dataframe when:
[also quoted from databricks](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html)


> - you want rich semantics, high-level abstractions, and domain specific APIs, use DataFrame
> - your processing demands high-level expressions, filters, maps, aggregation, averages, sum, SQL queries, columnar access and use of lambda functions on semi-structured data, use DataFrame
> - you want higher degree of type-safety at compile time, want typed JVM objects, take advantage of Catalyst optimization, and benefit from Tungsten’s efficient code generation, use Dataset.
> - you want unification and simplification of APIs across Spark Libraries, use DataFrame or Dataset.
> - If you are a R user, use DataFrames.
> - If you are a Python user, use DataFrames and resort back to RDDs if you need more control.

**Note**: Machine learning algorithms are run on _DataFrames_

## Step 1: Create spark context

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

## Step 2: Create a pandas dataframe

In [5]:
!ls

 1800.csv
 grep
'Lecture 01 - Class Introduction.pdf'
'Lecture 02 - Distributed Filesystems.pdf'
'Lecture 03 - MapReduce and Cloud Computing.pdf'
'Lecture 03 - Map Reduce.ipynb'
'Lecture 04 - Intro to RDDs.ipynb'
'Lecture 04 - Intro to Spark and RDDs.pdf'
'Lecture 05 Spark Dataframes.ipynb'
'Lecture 05 - Structured API 1.pdf'
'Lecture 05 users.csv'
 max_prcp.py
 max_temp.py
 powerpoints
 __pycache__
 README.md
 warpeace.txt
 wc.py


In [6]:
!head 'Lecture 05 users.csv'

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


In [7]:
!cp 'Lecture 05 users.csv' users.csv

In [8]:
!head users.csv

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


In [9]:
!wc -l users.csv

943 users.csv


In [15]:
import pandas as pd
p_df = pd.read_csv('users.csv', sep='|', header=None) 

In [17]:
p_df.columns=['id', 'age', 'gender', 'job', 'zip']

In [18]:
p_df

,id,age,gender,job,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


## Step 3: Create spark dataframe

In [21]:
rdd = sc.textFile("users.csv").map(lambda line: line.split("|"))

In [22]:
rdd.take(5)

[['1', '24', 'M', 'technician', '85711'],
 ['2', '53', 'F', 'other', '94043'],
 ['3', '23', 'M', 'writer', '32067'],
 ['4', '24', 'M', 'technician', '43537'],
 ['5', '33', 'F', 'other', '15213']]

In [23]:
df = spark.read.csv("users.csv", sep="|")

In [25]:
df.take(5)

[Row(_c0='1', _c1='24', _c2='M', _c3='technician', _c4='85711'),
 Row(_c0='2', _c1='53', _c2='F', _c3='other', _c4='94043'),
 Row(_c0='3', _c1='23', _c2='M', _c3='writer', _c4='32067'),
 Row(_c0='4', _c1='24', _c2='M', _c3='technician', _c4='43537'),
 Row(_c0='5', _c1='33', _c2='F', _c3='other', _c4='15213')]

In [26]:
# alternatively:
from_pandas_df = spark.createDataFrame(p_df)

In [27]:
from_pandas_df.show(5)

+---+---+------+----------+-----+
| id|age|gender|       job|  zip|
+---+---+------+----------+-----+
|  1| 24|     M|technician|85711|
|  2| 53|     F|     other|94043|
|  3| 23|     M|    writer|32067|
|  4| 24|     M|technician|43537|
|  5| 33|     F|     other|15213|
+---+---+------+----------+-----+
only showing top 5 rows



In [28]:
type(from_pandas_df)

pyspark.sql.dataframe.DataFrame

In [29]:
df.take(5)

[Row(_c0='1', _c1='24', _c2='M', _c3='technician', _c4='85711'),
 Row(_c0='2', _c1='53', _c2='F', _c3='other', _c4='94043'),
 Row(_c0='3', _c1='23', _c2='M', _c3='writer', _c4='32067'),
 Row(_c0='4', _c1='24', _c2='M', _c3='technician', _c4='43537'),
 Row(_c0='5', _c1='33', _c2='F', _c3='other', _c4='15213')]

In [30]:
# it's already a DF, but this is the easy way to rename columns
df = df.toDF("id", "age", "gender", "occupation", "zip")

In [31]:
df

DataFrame[id: string, age: string, gender: string, occupation: string, zip: string]

In [40]:
occupation_count = (
    df.where("occupation != 'other'")
      .groupby("occupation")
      .count()
      .sort("count", ascending=0)
)

In [68]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip: string (nullable = true)



In [65]:
(
df.where("occupation != 'other'")
  .groupby("occupation")
    .max()
)

DataFrame[occupation: string]

In [41]:
occupation_count.persist()

DataFrame[occupation: string, count: bigint]

In [42]:
occupation_count.show()

+-------------+-----+
|   occupation|count|
+-------------+-----+
|      student|  196|
|     educator|   95|
|administrator|   79|
|     engineer|   67|
|   programmer|   66|
|    librarian|   51|
|       writer|   45|
|    executive|   32|
|    scientist|   31|
|       artist|   28|
|   technician|   27|
|    marketing|   26|
|entertainment|   18|
|   healthcare|   16|
|      retired|   14|
|       lawyer|   12|
|     salesman|   12|
|         none|    9|
|    homemaker|    7|
|       doctor|    7|
+-------------+-----+



In [49]:
df.unpersist()

DataFrame[id: string, age: string, gender: string, occupation: string, zip: string]

In [47]:
df.limit(20).toPandas()
# df.head()
# df.show(5)

,id,age,gender,occupation,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [50]:
import pyspark.sql.functions as F
df.agg(F.countDistinct('occupation')).show()

+-----------------+
|count(occupation)|
+-----------------+
|               21|
+-----------------+



In [51]:
# We can also use SQL to query
df.createOrReplaceTempView('users')

query = """
SELECT occupation, COUNT(*) as count
FROM users
GROUP BY occupation
ORDER BY count DESC
"""
spark.sql(query).show(10)

+-------------+-----+
|   occupation|count|
+-------------+-----+
|      student|  196|
|        other|  105|
|     educator|   95|
|administrator|   79|
|     engineer|   67|
|   programmer|   66|
|    librarian|   51|
|       writer|   45|
|    executive|   32|
|    scientist|   31|
+-------------+-----+
only showing top 10 rows



In [55]:
occ_count_df = spark.sql('SELECT count(distinct occupation) as occ_count from users')

In [56]:
type(occ_count_df)

pyspark.sql.dataframe.DataFrame

## Chapter 5: Spark SQL and DataFrames: Interacting with External Data Sources
This notebook contains for code samples for *Chapter 5: Spark SQL and DataFrames: Interacting with External Data Sources*.

### User Defined Functions
While Apache Spark has a plethora of functions, the flexibility of Spark allows for data engineers and data scientists to define their own functions (i.e., user-defined functions or UDFs).

In [ ]:
from pyspark.sql.types import LongType

# Create cubed function
def cubed(s):
    return s * s * s

# Register UDF
spark.udf.register("cubed", cubed, LongType())

# Generate temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")

In [ ]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

## Speeding up and Distributing PySpark UDFs with Pandas UDFs
One of the previous prevailing issues with using PySpark UDFs was that it had slower performance than Scala UDFs.  This was because the PySpark UDFs required data movement between the JVM and Python working which was quite expensive.   To resolve this problem, pandas UDFs (also known as vectorized UDFs) were introduced as part of Apache Spark 2.3. It is a UDF that uses Apache Arrow to transfer data and utilizes pandas to work with the data. You simplify define a pandas UDF using the keyword pandas_udf as the decorator or to wrap the function itself.   Once the data is in Apache Arrow format, there is no longer the need to serialize/pickle the data as it is already in a format consumable by the Python process.  Instead of operating on individual inputs row-by-row, you are operating on a pandas series or dataframe (i.e. vectorized execution).

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import LongType

# Declare the cubed function
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a

# Create the pandas UDF for the cubed function 
cubed_udf = pandas_udf(cubed, returnType=LongType())

### Using pandas dataframe

In [ ]:
# Create a Pandas series
x = pd.Series([1, 2, 3])

# The function for a pandas_udf executed with local Pandas data
print(cubed(x))

In [ ]:
print(x.apply(cubed))

### Using Spark DataFrame

In [ ]:
# Create a Spark DataFrame
df = spark.range(1, 4)

from pyspark.sql.functions import col

# Execute function as a Spark vectorized UDF
df.select("id", cubed_udf(col("id"))).show()

## Higher Order Functions in DataFrames and Spark SQL

Because complex data types are an amalgamation of simple data types, it is tempting to manipulate complex data types directly. As noted in the post *Introducing New Built-in and Higher-Order Functions for Complex Data Types in Apache Spark 2.4* there are typically two solutions for the manipulation of complex data types.
1. Exploding the nested structure into individual rows, applying some function, and then re-creating the nested structure as noted in the code snippet below (see Option 1)  
1. Building a User Defined Function (UDF)

In [ ]:
# Create an array dataset
arrayData = [[1, (1, 2, 3)], [2, (2, 3, 4)], [3, (3, 4, 5)]]

# Create schema
from pyspark.sql.types import *
arraySchema = (StructType([
      StructField("id", IntegerType(), True), 
      StructField("values", ArrayType(IntegerType()), True)
      ]))

# Create DataFrame
df = spark.createDataFrame(spark.sparkContext.parallelize(arrayData), arraySchema)
df.createOrReplaceTempView("table")
df.printSchema()
df.show()

#### Option 1: Explode and Collect
In this nested SQL statement, we first `explode(values)` which creates a new row (with the id) for each element (`value`) within values.

In [ ]:
spark.sql("""
SELECT id, collect_list(value + 1) AS newValues
  FROM  (SELECT id, explode(values) AS value
        FROM table) x
 GROUP BY id
""").show()

#### Option 2: User Defined Function
To perform the same task (adding a value of 1 to each element in `values`), we can also create a user defined function (UDF) that uses map to iterate through each element (`value`) to perform the addition operation.

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import ArrayType

# Create UDF
def addOne(values):
    return [value + 1 for value in values]

# Register UDF
spark.udf.register("plusOneIntPy", addOne, ArrayType(IntegerType()))  

# Query data
spark.sql("SELECT id, plusOneIntPy(values) AS values FROM table").show()

### Higher-Order Functions
In addition to the previously noted built-in functions, there are high-order functions that take anonymous lambda functions as arguments.

In [ ]:
# In Python
from pyspark.sql.types import *
schema = StructType([StructField("celsius", ArrayType(IntegerType()))])

t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView("tC")

# Show the DataFrame
t_c.show()

#### Transform

`transform(array<T>, function<T, U>): array<U>`

The transform function produces an array by applying a function to each element of an input array (similar to a map function).

In [ ]:
# Calculate Fahrenheit from Celsius for an array of temperatures
spark.sql("""
SELECT
  celsius,
  transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit
FROM tC""").show()

#### Filter

`filter(array<T>, function<T, Boolean>): array<T>`

The filter function produces an array where the boolean function is true.

In [ ]:
# Filter temperatures > 38C for array of temperatures
spark.sql("""SELECT celsius, filter(celsius, t -> t > 38) as high FROM tC""").show()

#### Exists

`exists(array<T>, function<T, V, Boolean>): Boolean`

The exists function returns true if the boolean function holds for any element in the input array.

In [ ]:
# Is there a temperature of 38C in the array of temperatures
spark.sql("""
SELECT celsius, exists(celsius, t -> t = 38) as threshold
FROM tC
""").show()

#### Reduce

`reduce(array<T>, B, function<B, T, B>, function<B, R>)`

The reduce function reduces the elements of the array to a single value  by merging the elements into a buffer B using function<B, T, B> and by applying a finishing function<B, R> on the final buffer.

In [ ]:
# Calculate average temperature and convert to F
spark.sql("""
SELECT celsius, 
       reduce(
          celsius, 
          0, 
          (t, acc) -> t + acc, 
          acc -> (acc div size(celsius) * 9 div 5) + 32
        ) as avgFahrenheit 
  FROM tC
""").show()

## DataFrames and Spark SQL Common Relational Operators

The power of Spark SQL is that it contains many DataFrame Operations (also known as Untyped Dataset Operations). 

For the full list, refer to [Spark SQL, Built-in Functions](https://spark.apache.org/docs/latest/api/sql/index.html).

In the next section, we will focus on the following common relational operators:
* Unions and Joins
* Windowing
* Modifications

In [ ]:
from pyspark.sql.functions import expr

# Set File Paths
delays_path = "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
airports_path = "/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"

# Obtain airports dataset
airports = spark.read.options(header='true', inferSchema='true', sep='\t').csv(airports_path)
airports.createOrReplaceTempView("airports_na")

# Obtain departure Delays data
delays = spark.read.options(header='true').csv(delays_path)
delays = (delays
          .withColumn("delay", expr("CAST(delay as INT) as delay"))
          .withColumn("distance", expr("CAST(distance as INT) as distance")))

delays.createOrReplaceTempView("departureDelays")

# Create temporary small table
foo = delays.filter(expr("""
            origin == 'SEA' AND 
            destination == 'SFO' AND 
            date like '01010%' AND 
            delay > 0"""))

foo.createOrReplaceTempView("foo")

In [ ]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

In [ ]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

In [ ]:
spark.sql("SELECT * FROM foo LIMIT 10").show()

## Unions

In [ ]:
# Union two tables
bar = delays.union(foo)
bar.createOrReplaceTempView("bar")
bar.filter(expr("origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0")).show()

In [ ]:
spark.sql("""
SELECT * 
FROM bar 
WHERE origin = 'SEA' 
   AND destination = 'SFO' 
   AND date LIKE '01010%' 
   AND delay > 0
""").show()

## Joins
By default, it is an `inner join`.  There are also the options: `inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi, and left_anti`.

More info available at:
* [PySpark Join](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=join)

In [ ]:
# Join Departure Delays data (foo) with flight info
foo.join(
  airports, 
  airports.IATA == foo.origin
).select("City", "State", "date", "delay", "distance", "destination").show()

In [ ]:
spark.sql("""
SELECT a.City, a.State, f.date, f.delay, f.distance, f.destination 
  FROM foo f
  JOIN airports_na a
    ON a.IATA = f.origin
""").show()

## Windowing Functions

Great reference: [Introduction Windowing Functions in Spark SQL](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html)

> At its core, a window function calculates a return value for every input row of a table based on a group of rows, called the Frame. Every input row can have a unique frame associated with it. This characteristic of window functions makes them more powerful than other functions and allows users to express various data processing tasks that are hard (if not impossible) to be expressed without window functions in a concise way.

In [ ]:
spark.sql("DROP TABLE IF EXISTS departureDelaysWindow")
spark.sql("""
CREATE TABLE departureDelaysWindow AS
SELECT origin, destination, sum(delay) as TotalDelays 
  FROM departureDelays 
 WHERE origin IN ('SEA', 'SFO', 'JFK') 
   AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL') 
 GROUP BY origin, destination
""")

spark.sql("""SELECT * FROM departureDelaysWindow""").show()

What are the top three total delays destinations by origin city of SEA, SFO, and JFK?

In [ ]:
spark.sql("""
SELECT origin, destination, sum(TotalDelays) as TotalDelays
 FROM departureDelaysWindow
WHERE origin = 'SEA'
GROUP BY origin, destination
ORDER BY sum(TotalDelays) DESC
LIMIT 3
""").show()

In [ ]:
spark.sql("""
SELECT origin, destination, TotalDelays, rank 
  FROM ( 
     SELECT origin, destination, TotalDelays, dense_rank() 
       OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank 
       FROM departureDelaysWindow
  ) t 
 WHERE rank <= 3
""").show()

## Modifications

Another common DataFrame operation is to perform modifications to the DataFrame. Recall that the underlying RDDs are immutable (i.e. they do not change) to ensure there is data lineage for Spark operations. Hence while DataFrames themselves are immutable, you can modify them through operations that create a new, different DataFrame with different columns, for example.

### Adding New Columns

In [ ]:
foo2 = foo.withColumn("status", expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END"))
foo2.show()

In [ ]:
spark.sql("""SELECT *, CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END AS status FROM foo""").show()

### Dropping Columns

In [ ]:
foo3 = foo2.drop("delay")
foo3.show()

### Renaming Columns

In [ ]:
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

### Pivoting
Great reference [SQL Pivot: Converting Rows to Columns](https://databricks.com/blog/2018/11/01/sql-pivot-converting-rows-to-columns.html)

In [ ]:
spark.sql("""SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay FROM departureDelays WHERE origin = 'SEA'""").show(10)

In [ ]:
spark.sql("""
SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay 
  FROM departureDelays WHERE origin = 'SEA' 
) 
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) as AvgDelay, MAX(delay) as MaxDelay
  FOR month IN (1 JAN, 2 FEB, 3 MAR)
)
ORDER BY destination
""").show()

In [ ]:
spark.sql("""
SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay 
  FROM departureDelays WHERE origin = 'SEA' 
) 
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) as AvgDelay, MAX(delay) as MaxDelay
  FOR month IN (1 JAN, 2 FEB)
)
ORDER BY destination
""").show()

### Rollup
Refer to [What is the difference between cube, rollup and groupBy operators?](https://stackoverflow.com/questions/37975227/what-is-the-difference-between-cube-rollup-and-groupby-operators)